# Como treinar um modelo de NER usando BERT para reconheder entidades clínicas em português
## Fine Tuning com BERT para Reconhecimento de Entidades Nomeadas

### Introdução

Neste tutorial, faremos o *fine-tuning* ("ajuste fino") de um modelo BERT para tarefa de **Reconhecimento de entidade nomeadas (NER)**, para reconhecer entidades clínicas em português.

NER é uma tarefa de Processamento de Linguagem Natural (PLN) muito utilizada, servindo de base para outras tarefas, e consiste na identificação de entidades em dado não estruturado. Alguns exemplos de entidade nomeada são Pessoa, Localização, Número e Organização, mas os modelos também podem ser treinados para reconhecer entidades de um domínio específico.

#### Fluxo deste *notebook*

Dividimos o *notebook* nas seções: 

1. [Importando bibliotecas Python e preparando o ambiente](#section01)
2. [Importando e pré-processando os dados](#section02)
3. [Preparando o conjunto de dados e o Dataloader](#section03)
4. [Criação da rede neural para o *fine-tuning*](#section04)
5. [Treinando do modelo](#section05)
6. [Validando o desempenho do modelo](#section06)

#### Detalhes técnicos

Este script aproveita várias ferramentas e recursos desenvolvidos por outras equipes, listadas abaixo.
 
  - Objetivo:
- O objetivo deste script é utilizar **BertForTokenClassification** para treinar um modelo capaz de identificar as entidades clínicas, de acordo com o conjunto de dados fornecido. 
  - Dados:
- Vamos utilizar o conjunto de dados desse trabalho: [PortugueseClinicalNER](https://github.com/fabioacl/PortugueseClinicalNER/).
  - Modelo de linguagem:
- Vamos usar o [**BioBERTpt(all)**](https://github.com/HAILab-PUCPR/BioBERTpt) neste projeto, mas você pode usar o BERTimbau ou BERT-multilingual se desejar. 
- A equipe do Hugging Face criou uma classe customizada para classificação de tokens, chamada **BertForTokenClassification**. Mais informações na [Documentação](https://huggingface.co/transformers/model_doc/bert.html#bertfortokenclassification)
  - Requisitos:
- Python 3.6 ou superior
- Pytorch, Transformers e demaisbibliotecas Python de machine learning
- Preferência por GPU.

<a id='section01'> </a>
### Importando bibliotecas e preparando o ambiente

Vamos importar as bibliotecas necessárias para executar nosso script:
* Pandas
* Pytorch
* Pytorch Utils para dataset e Dataloader
* Transformers
* Modelo BERT e Tokenizer

*Caso você não possua a biblioteca seqeval (para cálculo das métricas), instale com o comando:*

```!pip -q install seqeval```

In [99]:
!pip -q install seqeval

You should consider upgrading via the 'c:\users\lisat\anaconda3\python.exe -m pip install --upgrade pip' command.


In [1]:
import torch
import numpy as np
import pandas as pd
import transformers
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertForTokenClassification, BertTokenizer, BertConfig, BertModel


<a id='section02'> </a>
### Importando e pré-processando os dados

Vamos preparar os dados. Você pode fazer *download* de todas as planilhas desse [repositório](https://github.com/fabioacl/PortugueseClinicalNER/tree/master/Texts%20SPN%201%20Labeled%20English), remover as colunas que não interessam e juntar tudo em um único arquivo. Felizmente, eu já fiz isso e deixei na pasta `data`.

* Vamos importar o arquivo em um dataframe 
* Vamos criar uma classe `GetSentenca` para criar frases com as palavras.
* Em seguida, vamos criar listas adicionais para usar em processamento futuro

In [2]:
dataset = pd.read_csv("./data/CorpusSaude.csv", sep=' ', names=['palavra','classe'])
dataset.head(15)

,palavra,classe
0,Durante,B-DT
1,internamento,I-DT
2,",",O
3,doente,O
4,teve,O
5,episodios,B-C
6,paroxisticos,I-C
7,de,O
8,diminuicao,B-C
9,de,I-C


In [3]:
#Adicionando uma coluna com os indices das frases
list_indices=[]
num_sentenca=1
for frase in dataset["palavra"]:
    #print(frase)
    list_indices.append(num_sentenca)
    if frase == '.':
        num_sentenca = num_sentenca+1
#dataset.head()
dataset['sentence_idx'] = list_indices

In [4]:
# Criando uma classe para formar frases a partir das palavras das colunas 
class GetSentenca(object):
    
    def __init__(self, dataset):
        self.n_sent = 1
        self.dataset = dataset
        self.empty = False
        agg_func = lambda s: [(w, t) for w, t in zip(s["palavra"].values.tolist(),
                                                       s['classe'].values.tolist())]
        self.grouped = self.dataset.groupby("sentence_idx").apply(agg_func)
        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None

getter = GetSentenca(dataset)

In [5]:
# Criação de listas e dicionários que serão usados posteriormente
tags_vals = list(set(dataset["classe"].values))
tag2idx = {t: i for i, t in enumerate(tags_vals)}
sentences = [' '.join([s[0] for s in sent]) for sent in getter.sentences]
labels = [[s[1] for s in sent] for sent in getter.sentences]
labels = [[tag2idx.get(l) for l in lab] for lab in labels]

In [6]:
# Verificando numero de classes únicas
print(len(set(tags_vals)))


25


In [23]:
# Rótulos, onde B sinaliza o inicio (BEGIN) e I, a continuação (INSIDE)
print(set(tags_vals))


{'B-CH', 'B-V', 'B-EV', 'B-AS', 'B-R', 'I-C', 'B-N', 'I-R', 'O', 'I-T', 'I-CH', 'B-THER', 'I-DT', 'B-T', 'I-V', 'B-OBS', 'B-RA', 'B-G', 'I-EV', 'I-THER', 'I-AS', 'I-OBS', 'B-C', 'I-G', 'B-DT'}


<a id='section03'> </a>
### Preparando o conjunto de dados e o Dataloader

Vamos definir algumas variáveis-chave que serão usadas no treinamento (*fine tuning*), e depois vamos criar a classe `Dataset`, que define como o texto é pré-processado antes de ser enviado para a rede neural. Vamos criar o `Dataloader` que envia os dados em lotes para a rede neural.

`Dataset` e `Dataloader` são classes da biblioteca `PyTorch` que ajudam a controlar o pré-processamento de dados e sua passagem no nosso modelo. Veja mais informações [aqui](https://pytorch.org/docs/stable/data.html).


#### * CustomDataset * Classe de conjunto de dados
- Esta classe recebe o `tokenizer`,` sentenças` e `rótulos` como entrada e gera como saída a sentença tokenizada com os rótulos, que serão usados no treinamento.
- Vamos usar um tokenizer baseado em BERT para tokenizar os dados da lista de `sentenças`.
- O tokenizer usa o método `encode_plus` para realizar a tokenização e gerar as saídas necessárias:` ids` e `attention_mask`. Veja mais informações sobre o tokenizer [aqui](https://huggingface.co/transformers/model_doc/bert.html#berttokenizer)
- Vamos criar dois conjuntos de dados, para treinamento e para validação
    - Conjunto de dados de treinamento, para treinar o modelo (**80% dos dados**)
    - Conjunto de dados de validação, para avaliar o desempenho do modelo (O modelo não viu esses dados durante o treinamento).

#### Dataloader
- Vamos usar a classe `Dataloader` para criar nosso *dataloader* de treinamento e validação, que carrega dados para a rede neural de uma maneira definida. Isso é necessário porque todos os dados do conjunto de dados não podem ser carregados para a memória de uma vez, portanto, a quantidade de dados carregados para a memória e enviados ao modelo precisa ser controlada.
- Este controle é obtido usando parâmetros como `batch_size` e `max_len`.

In [43]:
# Definindo variáveis que serão usadas posteriormente no treinamento

#MAX_LEN = 512
MAX_LEN = 200
#TRAIN_BATCH_SIZE = 32
#VALID_BATCH_SIZE = 16
TRAIN_BATCH_SIZE = 4
VALID_BATCH_SIZE = 4
EPOCHS = 3 # mude o número de épocas (sugestão = 10) 
LEARNING_RATE = 3e-05
# Vamos usar o tokenizador do BioBERTpt, próprio para dados clínicos / biomédicos
tokenizer = BertTokenizer.from_pretrained('pucpr/biobertpt-all')

In [44]:
class CustomDataset(Dataset):
    def __init__(self, tokenizer, sentences, labels, max_len):
        self.len = len(sentences)
        self.sentences = sentences
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len
        
    def __getitem__(self, index):
        sentence = str(self.sentences[index])
        inputs = self.tokenizer.encode_plus(
            sentence,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        label = self.labels[index]
        label.extend([4]*200)
        label=label[:200]

        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'tags': torch.tensor(label, dtype=torch.long)
        } 
    
    def __len__(self):
        return self.len

In [45]:
# Criação do conjunto de dados e dataloaders

train_percent = 0.8
train_size = int(train_percent*len(sentences))
# train_dataset=df.sample(frac=train_size,random_state=200).reset_index(drop=True)
# test_dataset=df.drop(train_dataset.index).reset_index(drop=True)
train_sentences = sentences[0:train_size]
train_labels = labels[0:train_size]

test_sentences = sentences[train_size:]
test_labels = labels[train_size:]

print("Tamanho do Dataset (total): {}".format(len(sentences)))
print("Tamanho do Dataset (treinamento): {}".format(len(train_sentences)))
print("Tamanho do Dataset (teste): {}".format(len(test_sentences)))

training_set = CustomDataset(tokenizer, train_sentences, train_labels, MAX_LEN)
testing_set = CustomDataset(tokenizer, test_sentences, test_labels, MAX_LEN)

Tamanho do Dataset (total): 1941
Tamanho do Dataset (treinamento): 1552
Tamanho do Dataset (teste): 389


In [46]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

<a id='section04'> </a>
### Criando nossa rede neural para treinarmos (*fine tuning*)

#### Rede neural
 - Vamos criar nossa rede neural com a classe `BERTClass`.
 - Como é uma tarefa de NER (*token-level*), nosso modelo será da classe `BertForTokenClassification`.
 - Os dados serão enviados à classe `BertForTokenClassification`, conforme definido no conjunto de dados.
 - Os resultados da camada final serão usados para calcular a perda e determinar a precisão dos modelos.
 - vamos criar uma instância da classe `model`, que será usada para treinamento e, em seguida, para salvar o modelo final  para inferência futura.
 
#### Função de perda e otimizador
 - Na sequencia, vamos definir um `Optimizer`, para atualizar os pesos da rede para melhorar seu desempenho.
 - Não vamos definir nenhuma função de perda, pois o modelo especificado já calcula a perda (`loss`) para uma determinada entrada.


In [47]:
model = transformers.BertForTokenClassification.from_pretrained('pucpr/biobertpt-all', num_labels=25) 

Some weights of the model checkpoint at pucpr/biobertpt-all were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at pucpr/biobertpt-all and are newly ini

In [48]:
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

<a id='section05'> </a>
### *Fine Tuning* - Ajustando o modelo

Depois de carregar e preparar os dados e conjuntos de dados e de criar o modelo e definir sua perda e otimizador, agora é hora de fazer o *fine-tuning* do modelo.

Vamos criar uma função para treinar o modelo no conjunto de dados de treinamento pelo número de vezes especificado (*EPOCH*). Uma época define quantas vezes os dados completos serão passados pela rede.

Os seguintes eventos acontecem no *fine-tuning*:
- O *dataloader* passa os dados para o modelo com base no tamanho do lote.
- A saída subsequente do modelo e a classe real são comparadas para calcular a perda.
- O valor de perda é usado para otimizar os pesos dos neurônios na rede.
- A cada 500 passos, o valor da perda é impresso no console.

Como você pode ver abaixo, em apenas 3 épocas o modelo estava com uma perda de 3.217659711837768. Você pode alterar o valor para treinar com mais épocas, para ver se o modelo está diminuindo a perda ao longo das épocas.


In [49]:
def train(epoch):
    model.train()
    for _,data in enumerate(training_loader, 0):
        ids = data['ids']
        mask = data['mask']
        targets = data['tags']
        
        ids = ids.type(torch.LongTensor)
        mask = mask.type(torch.LongTensor)
        targets = targets.type(torch.LongTensor)

        loss = model(input_ids=ids, token_type_ids=None, attention_mask=mask, labels=targets)[0]
        
        # optimizer.zero_grad()
        if _%500==0:
            print(f'Epoch: {epoch}, Loss:  {loss.item()}')
        
        optimizer.zero_grad()
        loss.backward()

In [50]:
for epoch in range(EPOCHS):
    train(epoch)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


RuntimeError: [enforce fail at ..\c10\core\CPUAllocator.cpp:72] data. DefaultCPUAllocator: not enough memory: you tried to allocate 201326592 bytes. Buy new RAM!


<a id='section06'> </a>
### Validando o modelo

Durante a validação, passamos os dados não vistos (ou seja, o conjunto de dados de teste) para o modelo. Esta etapa determina o quão bom é o desempenho do modelo nos dados não vistos.

Esses dados correspondem a 20% do nosso *dataset*, separado durante a criação do conjunto de dados.

Durante a fase de validação, os pesos do modelo não são atualizados. Apenas a saída final é comparada ao valor real. Essa comparação é então usada para calcular a precisão do modelo.

A métrica que vamos usar para medir o desempenho do modelo será a *F1-score*. Vamos criar uma função auxiliar para nos ajudar com o cálculo da pontuação F1 e também importar a biblioteca `seqeval`.

In [100]:
from seqeval.metrics import f1_score

def flat_accuracy(preds, labels):
    flat_preds = np.argmax(preds, axis=2).flatten()
    flat_labels = labels.flatten()
    return np.sum(flat_preds == flat_labels)/len(flat_labels)

In [105]:
def valid(model, testing_loader):
    model.eval()
    eval_loss = 0; eval_accuracy = 0
    n_correct = 0; n_wrong = 0; total = 0
    predictions , true_labels = [], []
    nb_eval_steps, nb_eval_examples = 0, 0
    with torch.no_grad():
        for _, data in enumerate(testing_loader, 0):
            ids = data['ids']
            mask = data['mask']
            targets = data['tags']

            output = model(ids, mask, labels=targets)
            loss, logits = output[:2]
            logits = logits.detach().cpu().numpy()
            label_ids = targets.to('cpu').numpy()
            predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
            true_labels.append(label_ids)
            accuracy = flat_accuracy(logits, label_ids)
            eval_loss += loss.mean().item()
            eval_accuracy += accuracy
            nb_eval_examples += ids.size(0)
            nb_eval_steps += 1
        eval_loss = eval_loss/nb_eval_steps
        print("Validation loss: {}".format(eval_loss))
        print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))
        pred_tags = [tags_vals[p_i] for p in predictions for p_i in p]
        valid_tags = [tags_vals[l_ii] for l in true_labels for l_i in l for l_ii in l_i]
        print("F1-Score: {}".format(f1_score([pred_tags], [valid_tags])))

In [106]:
# Para obter os resultados do conjunto de validação (dados não são vistos pelo modelo)

valid(model, testing_loader)

Validation loss: 3.235091218948364
Validation Accuracy: 0.14882749999999997
F1-Score: 0.18174698172038248


In [109]:
# salvando o modelo para usar mais tarde
OUTPUT_MODEL_PATH = './model' 
model.l1.save_pretrained(OUTPUT_MODEL_PATH)

In [120]:
def predict(model, testing_loader):
    model.eval()
    eval_loss = 0; eval_accuracy = 0
    n_correct = 0; n_wrong = 0; total = 0
    predictions , true_labels = [], []
    nb_eval_steps, nb_eval_examples = 0, 0
    with torch.no_grad():
        for _, data in enumerate(testing_loader, 0):
            ids = data['ids']
            mask = data['mask']
            targets = data['tags']

            output = model(ids, mask, labels=targets)
            loss, logits = output[:2]
            logits = logits.detach().cpu().numpy()
            label_ids = targets.to('cpu').numpy()
            predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
    print(predictions)

In [147]:
def predictBERTNER(sentencas, model, tokenizer):
    predictedModel=[]
    model.eval()
    for test_sentence in sentencas:
        tokenized_sentence = tokenizer.encode_plus(test_sentence)
        input_ids = tokenized_sentence['input_ids']
        input_masks = tokenized_sentence['attention_mask']
        
        with torch.no_grad():
            output = model(input_ids, input_masks)
        label_indices = np.argmax(output[0].to('cpu').numpy(), axis=2)
        
        # join bpe split tokens
        tokens = tokenizer.convert_ids_to_tokens(input_ids.to('cpu').numpy()[0])
        new_tokens, new_labels = [], []
        for token, label_idx in zip(tokens, label_indices[0]):
            if token.startswith("##"):
                new_tokens[-1] = new_tokens[-1] + token[2:]
            else:
                new_labels.append(label_idx)
                new_tokens.append(token)
            
        FinalLabelSentence = []
        for token, label in zip(new_tokens, new_labels):
            label = idx2tag[str(label)]
            if label == "O" or label == "X":
                FinalLabelSentence.append("O")
            else:
                FinalLabelSentence.append(label)
                
        predictedModel.append(FinalLabelSentence[1:-1]) # delete [SEP] and [CLS]
        
            
    return predictedModel

In [118]:
import nltk    
from nltk import tokenize    

# THE MODEL ACCEPTS ONLY LOWER
test_sentence1 = "Paciente com Sepse pulmonar em D8 tazocin (pciente não recebeu por 2 dias Atb).".lower()
test_sentence2 = "Acesso venoso central em subclavia D duplolumen recebendo solução salina e glicosada em BI.".lower()

test_sentence_tokenized = [tokenize.word_tokenize(test_sentence1, language='portuguese'),tokenize.word_tokenize(test_sentence2, language='portuguese')] 
print(test_sentence_tokenized)

[['paciente', 'com', 'sepse', 'pulmonar', 'em', 'd8', 'tazocin', '(', 'pciente', 'não', 'recebeu', 'por', '2', 'dias', 'atb', ')', '.'], ['acesso', 'venoso', 'central', 'em', 'subclavia', 'd', 'duplolumen', 'recebendo', 'solução', 'salina', 'e', 'glicosada', 'em', 'bi', '.']]


In [148]:
# Carregando o modelo treinado e seu vocabulário
#model = model_class.from_pretrained(OUTPUT_MODEL_PATH)
#tokenizer = tokenizer_class.from_pretrained(OUTPUT_MODEL_PATH)

tags = predictBERTNER(test_sentence_tokenized, model, tokenizer)
tags


TypeError: forward() missing 1 required positional argument: 'labels'